In [7]:
import os, warnings
warnings.filterwarnings('ignore')
# NVIDIA SETTINGS 
# Please configure according to the situation of your own device
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import tensorflow as tf
import gc
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

import pandas as pd
import numpy as np

from joblib import load, dump

def r2_score(y_true,y_pred):
    y_mean = np.mean(y_true)
    r2 = 1-sum((y_true-y_pred)**2)/sum((y_mean-y_true)**2)
    return r2

def PCC(y_pred,y_true):
    diff_pred,diff_true=y_pred-np.mean(y_pred),y_true-np.mean(y_true)
    return np.sum(diff_pred*diff_true)/np.sqrt(np.sum(diff_pred**2)*np.sum(diff_true**2))

def to_bav(x, max_bav=50000.0):
    return max_bav ** (1.0 - x)

sample_weight=None,
threshold_nm=500,
max_bav=50000

read `blosum62 matrix` and `pam250 matrix`

In [8]:
blosum62 = pd.read_csv('../blosum_pam_data/BLOSUM62.txt', sep='\s')
blosum62 = blosum62.iloc[:-4,:-4]

pam250 = pd.read_csv('../blosum_pam_data/PAM250.csv',index_col=0)

In [9]:
#AApro normalization to 0~1
PP = pd.read_csv('../physicochemical_properties/PP_61.csv', index_col='properties').index
AApro_dic = pd.read_csv('../umap/single_index/AApro_UMAP.csv', header=0,index_col=0)
AA_pro = AApro_dic.loc[PP].astype('float')
AA_pro = AA_pro.T
AA_pro = ((AA_pro - AA_pro.min()) / (AA_pro.max() - AA_pro.min())).T #normalization

Generation and splicing of sequence correlation aapro, blosum62, pam250 distance matrix

In [10]:
def get_3d_feat(seq):
    n = len(seq)

    # Generate physical and chemical property matrix
    seq_pro = pd.DataFrame(AA_pro[aa] for aa in seq).T
    seq_pro = seq_pro.values[:, :, None]  # Two dimensions become three dimensions (61, n, 1)
    # Multiply
    mt_pro = np.transpose((seq_pro * np.transpose(seq_pro, [0, 2, 1])), [1, 2, 0])
    # transpose three-dimensional transposition. (61,n,1)*(61,1,n)=(61,n,n), and then transpose to (n,n,61).

    # Generate blosum matrix
    seq_blosum = np.ones((n,n))
    for i in range(n):
        a = seq[i]
        for j in range(n):
            b = seq[j]
            seq_blosum[i][j] = blosum62.loc[a,b]
    # Normalize the blosum62 matrix to 0~1
    seq_blosum_nor = ((seq_blosum - seq_blosum.min()) / (seq_blosum.max() - seq_blosum.min()))
    seq_blosum_nor = seq_blosum_nor[:, :, np.newaxis]  #(n,n,1)
    
    # Generate pam matrix
    seq_pam = np.ones((n,n))
    for i in range(n):
        a = seq[i]
        for j in range(n):
            b = seq[j]
            seq_pam[i][j] = pam250.loc[a,b]
    # Normalize the pam250 mutation matrix to 0~1
    seq_pam_nor = ((seq_pam - seq_pam.min()) / (seq_pam.max() - seq_pam.min()))
    seq_pam_nor = seq_pam_nor[:, :, np.newaxis]  # (n,n,1)
    
    # Generate sequence distance matrix
    pt_dis = np.ones((n,n))
    for i in range(n):
        for j in range(n):
            pt_dis[i][j] = abs(i-j)
    pt_dis = ((pt_dis - 0) / (n-1 - 0)) # Normalization (minimum value is 0, maximum value is n-1)
    pt_dis = pt_dis[:, :, np.newaxis]  # (n,n,1)

    #Change the lower half of the physical and chemical property matrix
    for k in range(mt_pro.shape[2]):
        for i in range(n):
            for j in range(i):
                if k < 60:                                      
                    mt_pro[i,j,k] = (mt_pro[j,i,k] * mt_pro[j,i,k+1])**.5
                else:
                    mt_pro[i,j,k] = (mt_pro[j,i,k] * mt_pro[j,i,0])**.5

    # Merge the sequence distance matrix with mt and place it on the first layer
    mt = np.concatenate((pt_dis,seq_blosum_nor,seq_pam_nor,mt_pro),axis = 2)

    #Fill with 0 to Make the xshape corresponding to each sequence in the data set the same.
    x = np.pad(mt, [(0, max_seq_len-n), (0, max_seq_len-n), (0, 0)]) 

    return x[ :, :, :, None]

predict on ZSbenchmark

In [11]:
Test_dir = '../../calculate_webtools_score/ZSbenchmark/ZSbenchmark_iedb_tools_predict_data_processed'
tf_model_dir = 'tf_model'

for allele in os.listdir(Test_dir):
    if allele not in os.listdir(tf_model_dir):
        print(f'{allele} is not supportable')
        continue
    
    df_test = pd.read_csv(os.path.join(Test_dir, allele, 'tools_processed.csv'))  
    
    print(f'*************{allele} predict start**************')
    
    middle_data_dir = '../../calculate_webtools_score/ZSbenchmark/ZSbenchmark_VRAPERNet_pred_middle_data'
    allele_fold = os.path.join(middle_data_dir, allele)

    if not os.path.exists(allele_fold) : 
        os.makedirs(allele_fold)
        
    max_seq_len = 15
    #HLA_0205 11
    #HLA_3201 13
    #HLA_2704 12
    #HLA-B_2706 12
    #HLB-B_3801 11
   
    # Generate middle data X_test
    X_test_name = os.path.join(allele_fold, allele+'_X_test_'+'.data')
    if not os.path.exists(X_test_name) :
        X_test = []
        for seq in df_test['peptide']:
            X_test.append(get_3d_feat(seq))
        X_test = np.stack(X_test)
        dump(X_test, X_test_name)
    else:
        X_test = load(X_test_name)
    X_test = X_test.astype('float32')

    
    model = tf.keras.models.load_model(f'{tf_model_dir}/{allele}')
    # make prediction
    Y_test_pred = model.predict(X_test)
    df_pred = pd.DataFrame(Y_test_pred.tolist()).rename(columns={0:'Pred_Norm_QM'})
    df_pred['Pred_QM'] = df_pred['Pred_Norm_QM'].apply(to_bav)
    df_test['VRAPERNet_BAV_Normalized'] = df_pred['Pred_Norm_QM']
    df_test['VRAPERNet_BAV'] = df_pred['Pred_QM']
    df_test.to_csv(os.path.join(Test_dir, allele, 'tools_processed.csv'), index=False)
    
    del model
 
    print(f'*************{allele} predict finished**************')
gc.collect()


*************HLA-A_0101 predict start**************
2/2 [==============================] - 0s 6ms/step
*************HLA-A_0101 predict finished**************
*************HLA-A_0201 predict start**************
14/14 [==============================] - 0s 9ms/step
*************HLA-A_0201 predict finished**************
*************HLA-A_0202 predict start**************
5/5 [==============================] - 0s 8ms/step
*************HLA-A_0202 predict finished**************
*************HLA-A_0203 predict start**************
4/4 [==============================] - 0s 15ms/step
*************HLA-A_0203 predict finished**************
*************HLA-A_0205 predict start**************
1/1 [==============================] - 0s 118ms/step
*************HLA-A_0205 predict finished**************
*************HLA-A_0206 predict start**************
3/3 [==============================] - 0s 7ms/step
*************HLA-A_0206 predict finished**************
*************HLA-A_0301 predict start**********

54122

predict on IEDB 2022-2023 new-release

In [12]:
Test_dir = '../../calculate_webtools_score/IEDB_new_released_dataset/after_2022_iedb_tools_predict_data_processed'
tf_model_dir = 'tf_model'

for allele in os.listdir(Test_dir):
    if allele not in os.listdir(tf_model_dir):
        print(f'{allele} is not supportable')
        continue
    
    df_test = pd.read_csv(os.path.join(Test_dir, allele, 'tools_processed.csv'))  
    
    print(f'*************{allele} predict start**************')
    
    middle_data_dir = '../../calculate_webtools_score/IEDB_new_released_dataset/after_2022_VRAPERNet_pred_middle_data'
    allele_fold = os.path.join(middle_data_dir, allele)

    if not os.path.exists(allele_fold) : 
        os.makedirs(allele_fold)
    max_seq_len = 15
   
    # Generate middle data X_test
    X_test_name = os.path.join(allele_fold, allele+'_X_test_'+'.data')
    if not os.path.exists(X_test_name) :
        X_test = []
        for seq in df_test['peptide']:
            X_test.append(get_3d_feat(seq))
        X_test = np.stack(X_test)
        dump(X_test, X_test_name)
    else:
        X_test = load(X_test_name)
    X_test = X_test.astype('float32')

    
    model = tf.keras.models.load_model(f'{tf_model_dir}/{allele}')
    # make prediction
    Y_test_pred = model.predict(X_test)
    df_pred = pd.DataFrame(Y_test_pred.tolist()).rename(columns={0:'Pred_Norm_QM'})
    df_pred['Pred_QM'] = df_pred['Pred_Norm_QM'].apply(to_bav)
    df_test['VRAPERNet_BAV_Normalized'] = df_pred['Pred_Norm_QM']
    df_test['VRAPERNet_BAV'] = df_pred['Pred_QM']
    df_test.to_csv(os.path.join(Test_dir, allele, 'tools_processed.csv'), index=False)
    
    del model
 
    print(f'*************{allele} predict finished**************')
gc.collect()


*************HLA-A_0201 predict start**************
3/3 [==============================] - 0s 8ms/step
*************HLA-A_0201 predict finished**************
*************HLA-A_1101 predict start**************
1/1 [==============================] - 0s 84ms/step
*************HLA-A_1101 predict finished**************
*************HLA-A_2402 predict start**************
1/1 [==============================] - 0s 148ms/step
*************HLA-A_2402 predict finished**************


59955